In [47]:
# Import the necessary dependencies
import numpy as np
import matplotlib.pyplot as plt

# Initialize physical constants
k_B = 1.38e-23  #  [J/K]
T = 10000  # [K]
mass_e = 9.11e-31  # [kg]

def initialize_velocities(T, mass, num_particles):
    std_dev_velocity = np.sqrt(k_B * T / mass)
    vx = np.random.normal(0, std_dev_velocity, num_particles)
    vy = np.random.normal(0, std_dev_velocity, num_particles)
    return vx, vy

# Create the Grid's class
class Grid:
    def __init__(self, Lx, Ly, nx, ny, Bz, potential_upper, potential_lower):
        """
        Initialize a 2D Grid with periodic x-boundary, constant B and potential

        Args:
            x_size (float): Physical size of the grid in the x-direction.
            y_size (float): Physical size of the grid in the y-direction.
            dx (float) : grid spacing in the x-direction
            dy (float) : grid spacing in the y-direction
            Bz (float) : Constant magnetic field (out of the plane)
            potential (float) : Fixed potential
        """
        self.Lx = Lx
        self.Ly = Ly

        self.nx = nx
        self.ny = ny

        self.dx = dx
        self.dy = dy
        
        # Initialize the fields
        self.E = np.zeros((self.nx, self.ny, 2)) # Ex and Ey
        self.B = np.zeros((self.nx, self.ny, 1)) # Bz
        self.B[:, :, -1] = Bz # Set a constant Bz

        # Initialize the potential
        self.phi = np.zeros((self.nx+1, self.ny+1, 1))
        self.phi[:, -1] = potential_upper # Set the first upper row to a constant potential
        self.phi[:, 0] = potential_lower # Set the first lower row to a constant potential

    def poisson_solver(self, epsilon_0, tol=1e-5, max_iter=10000):
        """
        Solve the Poisson equation using Successive Over-Relaxation (SOR).
        
        Args:
            epsilon_0 (float): Permittivity of free space.
            tol (float): Convergence tolerance.
            max_iter (int): Maximum number of iterations.
        """
        omega = 1.83  # Over-relaxation factor (1 < omega < 2)
        dx2 = self.dx**2
        dy2 = self.dy**2
        denom = 2 * (dx2 + dy2)
        
        for _ in range(max_iter):
            old_phi = self.phi.copy()
            
            # Update potential using SOR
            for i in range(1, self.nx):
                for j in range(1, self.ny):
                    self.phi[i, j] = (1 - omega) * self.phi[i, j] + omega * ((self.phi[i+1, j] + self.phi[i-1, j]) * dy2 +
                                                                             (self.phi[i, j+1] + self.phi[i, j-1]) * dx2) / denom
                    """ This is for the case where the particles interact with the electric field
                    self.phi[i, j] = (1 - omega) * self.phi[i, j] + omega * ((self.phi[i+1, j] + self.phi[i-1, j]) * dy2 +
                                                                             (self.phi[i, j+1] + self.phi[i, j-1]) * dx2 -
                                                                             self.rho[i, j] * dx2 * dy2 / epsilon_0) / denom
                                                                             """
            
            # Enforce periodic boundary in x
            self.phi[0, :] = self.phi[-2, :]
            self.phi[-1, :] = self.phi[1, :]
            
            # Check for convergence
            if np.max(np.abs(self.phi - old_phi)) < tol:
                break

    def calculate_E(self):
        """
        Calculate the electric field from the potential using finite differences
        """
        # Ex = -d(phi)/dx, Ey = -d(phi)/dy
        Ex = -(np.roll(self.phi, -1, axis=0) - np.roll(self.phi, 1, axis=0)) / (2 * self.dx)
        Ey = -(np.roll(self.phi, -1, axis=1) - np.roll(self.phi, 1, axis=1)) / (2 * self.dy)
        
        self.E[:, :, 0] = Ex  # Assign Ex to the grid
        self.E[:, :, 1] = Ey  # Assign Ey to the grid


# Create the particles class
class Particle:
    def __init__(self, x, y, vx, vy, charge, mass):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.charge = charge
        self.mass = mass

In [48]:
epsilon_0 = 8.854e-12  # Permittivity of free space

# Boris push function
def interpolate_field(field, x, y, dx, dy):
    """
    Perform bilinear interpolation to calculate the field value at a particle's position.

    Args:
        field (np.ndarray): 2D field array.
        x, y (float): Particle position.
        dx, dy (float): Grid spacing.

    Returns:
        float: Interpolated field value.
    """
    # Convert positions to grid indices
    i = int(x / dx)
    j = int(y / dy)
    rmodx = (x - i * dx) / dx
    rbody = (y - j * dy) / dy

    # Ensure indices are within the array bounds
    i_plus = min(i + 1, field.shape[0] - 1)
    j_plus = min(j + 1, field.shape[1] - 1)

    # Bilinear interpolation
    interpolated_value = (
        (1 - rmodx) * (1 - rbody) * field[i, j] +
        rmodx * (1 - rbody) * field[i_plus, j] +
        (1 - rmodx) * rbody * field[i, j_plus] +
        rmodx * rbody * field[i_plus, j_plus]
    )

    return interpolated_value

def boris_push(particles, efield_X, efield_Y, Bz, dt, dx, dy):
    """
    Perform the Boris push to update the positions and velocities of particles.

    Args:
        particles (list of Particle): List of particles.
        efield_X, efield_Y (np.ndarray): Electric field components on the grid.
        Bz (float): Constant magnetic field (out of the plane).
        dt (float): Time step.
        dx, dy (float): Grid spacing.
    """
    for particle in particles:
        # First half leapfrog position update
        particle.x += particle.vx * (0.5 * dt)
        particle.y += particle.vy * (0.5 * dt)

        # Interpolate electric field at particle position
        ex = interpolate_field(efield_X, particle.x, particle.y, dx, dy)
        ey = interpolate_field(efield_Y, particle.x, particle.y, dx, dy)

        # First half electric field acceleration
        particle.vx += ex * particle.q * dt / (2.0 * particle.m)
        particle.vy += ey * particle.q * dt / (2.0 * particle.m)

        # Boris rotation for magnetic field
        t_x = particle.q * Bz * dt / (2.0 * particle.m)
        t_y = -t_x  # Negative for right-hand rule in 2D
        s = 2 * t_x / (1 + t_x * t_x + t_y * t_y)
        v_prime_x = particle.vx + particle.vy * t_x
        v_prime_y = particle.vy - particle.vx * t_y
        particle.vx += v_prime_y * s
        particle.vy -= v_prime_x * s

        # Second half electric field acceleration
        particle.vx += ex * particle.q * dt / (2.0 * particle.m)
        particle.vy += ey * particle.q * dt / (2.0 * particle.m)

        # Second half leapfrog position update
        particle.x += particle.vx * (0.5 * dt)
        particle.y += particle.vy * (0.5 * dt)

In [49]:
import numpy as np

dt = 0.01 # Time step [s]
t = 1. # Total time [s]
Nt = t / dt # Number of time steps

potential_lower = 0.  # [V]
potential_upper = 100. # [V]
Bz = 1 # [T]

num_particles = 2
T = 10000 # Temperature [K]
m = 9.11e-31 # electron's mass [kg]
charge = 1.602e-19 # elemental charge [C]
Lx = 10. # [m]
Ly = 10. # [m]
Nx = int(20) # Number of bins in x-direction
Ny = int(20) # Number of bins in y-direction
dx = Lx / Nx
dy = Ly / Ny

# Initiliaze the grid
grid = Grid(Lx, Ly, Nx, Ny, Bz, potential_upper, potential_lower)

vx, vy = initialize_velocities(T, m, num_particles)
particles = [
    Particle(
        x = np.random.uniform(0, Lx),
        y = np.random.uniform(0, Ly),
        vx = vx[i],
        vy = vy[i],
        charge = charge,
        mass = m
    ) for i in range(num_particles)
]

grid.Lx

10.0

In [52]:
# Calculate the potential using the poisson solver
grid.poisson_solver(epsilon_0)

# Calculate the electric field
# grid.calculate_E()

# Check the electric field
grid.phi

array([[[  0.        ],
        [  5.00000348],
        [  9.99999998],
        [ 15.00000165],
        [ 19.99999892],
        [ 25.00000166],
        [ 29.99999884],
        [ 35.00000303],
        [ 39.99999652],
        [ 45.00000341],
        [ 49.99999803],
        [ 54.99999948],
        [ 59.99999846],
        [ 64.99999864],
        [ 70.00000051],
        [ 74.99999712],
        [ 80.00000122],
        [ 84.99999912],
        [ 90.00000054],
        [ 94.99999792],
        [100.        ]],

       [[  0.        ],
        [  4.99999659],
        [ 10.00000021],
        [ 14.99999843],
        [ 20.00000147],
        [ 24.99999843],
        [ 30.00000161],
        [ 34.99999701],
        [ 40.00000398],
        [ 44.99999662],
        [ 50.0000023 ],
        [ 55.00000066],
        [ 60.00000181],
        [ 65.0000016 ],
        [ 69.99999962],
        [ 75.00000292],
        [ 79.99999906],
        [ 85.00000061],
        [ 89.99999953],
        [ 95.00000187],
        [100. 

In [42]:
for i in range(1, Ny):
    for j in range(1, Nx):
        print(i, j)

1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
6 16
6 17
6 18
6 19
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
7 15
7 16
7 17
7 18
7 19
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
8 13
8 14
8 15
8 16
8 17
8 18
8 19
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
9 12
9 13
9 14
9 15
9 16
9 17
9 18
9 19
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
10 12
10 13
10 14
10 15
10 16
10 17
10 18
10 19
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11
11 12
11 13
11 14
11 15
11 16
11 17
11 18
11 19
12 1
12 2
12 3
1